In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import os

2025-03-20 12:39:21.851819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742441961.876385     345 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742441961.883827     345 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742441961.903679     345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742441961.903700     345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742441961.903703     345 computation_placer.cc:177] computation placer alr

In [10]:
import kagglehub

path = kagglehub.dataset_download("sobhanmoosavi/us-road-construction-and-closures")
print("Path to dataset files:", path)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 760M/760M [00:26<00:00, 30.6MB/s] 

Extracting files...


Path to dataset files: /home/work/.cache/kagglehub/datasets/sobhanmoosavi/us-road-construction-and-closures/versions/1


In [12]:
def get_data(path):
    try:
        # Load dataset
        df = pd.read_csv(path)
    except FileNotFoundError:
        return "File not found"
    except Exception as e:
        return f"An error occurred while loading the dataset: {e}"

    # Convert time columns to datetime and compute duration (in minutes)
    def parse_datetime(dt_str):
      # Try formats with microseconds first, then without
      for fmt in ("%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d %H:%M:%S"):
          try:
              return pd.to_datetime(dt_str, format=fmt)
          except ValueError:
              continue
      # Return NaT if none of the formats work
      return pd.NaT

    # Apply the custom function to the relevant columns
    df['Start_Time'] = df['Start_Time'].apply(parse_datetime)
    df['End_Time'] = df['End_Time'].apply(parse_datetime)

    # Drop rows where the datetime conversion failed
    df = df.dropna(subset=['Start_Time', 'End_Time'])

    df['Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60.0

    # Define the feature columns
    features = [
        'Distance(mi)',
        'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng',
        'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
        'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
        'Precipitation(in)',
        'Duration'
    ]

    # Drop rows that have missing values in any of the selected features or in the target
    df = df[['Severity'] + features].dropna()

    # Encode Severity into integer classes (0, 1, 2, 3)
    # This step will automatically map the unique severity values (e.g., 1, 2, 3, 4) to 0,1,2,3.
    le = LabelEncoder()
    df['Severity_encoded'] = le.fit_transform(df['Severity'])

    # Prepare input features and target
    X = df[features].values.astype(np.float32)
    y = df['Severity_encoded'].values.astype(np.int32)

    # Standardize features for better NN performance
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

In [23]:
def train_nn(X_train, y_train):
    # Build the neural network model
    model = Sequential([
        Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=8192, validation_split=0.1, verbose=1)

    return model
    

In [18]:
csv = os.path.join(path, 'US_Constructions_Dec21.csv')
X_train, X_test, y_train, y_test = get_data(csv)

In [24]:
model = train_nn(X_train, y_train)

Epoch 1/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7660 - loss: 0.7405 - val_accuracy: 0.8871 - val_loss: 0.3969
Epoch 2/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8867 - loss: 0.3951 - val_accuracy: 0.8871 - val_loss: 0.3872
Epoch 3/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8867 - loss: 0.3860 - val_accuracy: 0.8871 - val_loss: 0.3797
Epoch 4/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8870 - loss: 0.3777 - val_accuracy: 0.8872 - val_loss: 0.3673
Epoch 5/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8871 - loss: 0.3640 - val_accuracy: 0.8895 - val_loss: 0.3524
Epoch 6/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8897 - loss: 0.3502 - val_accuracy: 0.8920 - val_loss: 0.3436
Epoch 7/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8918 - loss: 0.3428 - val_accuracy: 0.8928 - val_loss: 0.3389
Epoch 8/50
414/414 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8925 - loss: 0.3385 - val_accuracy: 0.

In [25]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Test loss: 0.3034529685974121
Test accuracy: 0.9026808142662048
